In [1]:
import pandas as pd
import matplotlib.pyplot as plt  
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Load dataset)
#Importing Required Libraries
import sys
import os

# Import the python libraies from source
#Allows for model to reload without reloading Kernel
import importlib 

#Python files can be used 
sys.path.append(os.path.abspath("../src"))

#prepare_data.py file importing functions
from prepare_data import (
    load_data,
    catagorical_column_transformations,
    feature_engineering
    
)
#preprocess_data.py file importing functions
from preprocess_data import (
    scale_and_encode,
    remove_anomalies,
    get_numeric_columns,
    iterative_outlier_removal,
    make_Onehot
)
#train_models.py file importing functions
from train_models import (
    split_features_target,
    create_train_test_split,
    get_model,
    train_model
)


# Data preparation
# load the data using the load_data function from prepare_data.py
df = load_data('..\data\Student_performance_data.csv')
# decode the catagorical features
df = catagorical_column_transformations(df)
# perform feature engineering using the feature_engineering function from prepare_data.py
df = feature_engineering(df)
numeric_columns = get_numeric_columns(df)
df = scale_and_encode(df)
# iterative outlier removal using the iterative_outlier_removal function from preprocess_data.py
df = iterative_outlier_removal(df, numeric_columns)
df = make_Onehot(df)
X, y = split_features_target(df)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an XGBoost DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for the XGBoost model
params = {
    'objective': 'multi:softmax',  # Specify multiclass classification
    'num_class': 1,                 # Number of classes in the target variable
    'max_depth': 3,                 # Maximum depth of a tree
    'eta': 0.1,                     # Learning rate
    'eval_metric': 'mlogloss'       # Evaluation metric
}

# Train the model
num_round = 100  # Number of boosting rounds
bst = xgb.train(params, dtrain, num_round)

# Make predictions
y_pred = bst.predict(dtest)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

#Grade mapping
grade_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'F'}
y_test_letters = pd.Series(y_test).map(grade_map)
y_pred_letters = pd.Series(y_pred).map(grade_map)
label_names = [grade_map[i] for i in sorted(grade_map)]

#Step 8: Evaluation
accuracy = accuracy_score(y_test_letters, y_pred_letters)
print("XGBoost Accuracy:", round(accuracy, 4))
print("Classification Report:")
print(classification_report(y_test_letters, y_pred_letters, target_names=label_names, zero_division=0))

#Step 11: Save the model as pkl file in artifacts

with open("../artifacts/xgboost_model.pkl", "wb") as file:
   pickle.dump(bst, file)

#Step 12: Save predictions to CSV
comp_df = X_test.copy()
comp_df["Actual_GradeClass"] = y_test.values
comp_df["Predicted_GradeClass"] = y_pred

#Step 13: Show and save the predictions table
try:
    from IPython.display import display

    # Prepare DataFrame for display
    comp_df = pd.DataFrame({"Actual": y_test.values,"Predicted": y_pred})
    comp_df["Match"] = comp_df["Actual"] == comp_df["Predicted"]

    def highlight_false_text(row):
        styles = []
        for col in row.index:
            if col == "Match" and row["Match"] == False:
                styles.append("color: red; background-color: black")
            else:
                styles.append("background-color: black; color: white")
        return styles

    print("First 20 Predictions:")
    display(comp_df.head(20).style.apply(highlight_false_text, axis=1))

except Exception as e:
    print("\n First 20 Predictions:")
    print(comp_df.head(20).to_string(index=False))

comp_df.to_csv("../artifacts/xgboost_prediction.csv", index=False)

<>:54: SyntaxWarning: invalid escape sequence '\d'
<>:54: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Karl\AppData\Local\Temp\ipykernel_25560\2148462400.py:54: SyntaxWarning: invalid escape sequence '\d'
  df = load_data('..\data\Student_performance_data.csv')
c:\Users\Karl\source\repos\ApexCareSolutions\MLG382_Guided_Project_GroupX\src\preprocess_data.py:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({False: 0.0, True: 1.0})
C:\Users\Karl\AppData\Local\Temp\ipykernel_25560\2148462400.py:54: SyntaxWarning: invalid escape sequence '\d'
  df = load_data('..\data\Student_performance_data.csv')


XGBoostError: [17:53:10] C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\objective\multiclass_obj.cu:111: SoftmaxMultiClassObj: label must be in [0, num_class).

# XGBoost

In [1]:
#Importing Required Libraries
import sys
import os

# Import the python libraies from source
#Allows for model to reload without reloading Kernel
import importlib 

#Python files can be used 
sys.path.append(os.path.abspath("../src"))

#prepare_data.py file importing functions
from prepare_data import (
    load_data,
    catagorical_column_transformations,
    feature_engineering
    
)
#preprocess_data.py file importing functions
from preprocess_data import (
    scale_and_encode,
    remove_anomalies,
    get_numeric_columns,
    iterative_outlier_removal,
    make_Onehot
)
#train_models.py file importing functions
from train_models import (
    split_features_target,
    create_train_test_split,
    get_model,
    train_model
)


# Data preparation
# load the data using the load_data function from prepare_data.py
df = load_data('..\data\Student_performance_data.csv')
# decode the catagorical features
df = catagorical_column_transformations(df)
# perform feature engineering using the feature_engineering function from prepare_data.py
df = feature_engineering(df)
# Preprocessing
# scale and encode the data using the scale_and_encode function from preprocess_data.py
df = scale_and_encode(df)
#print(df.columns.to_list())
# remove anomalies using the remove_anomalies function from preprocess_data.py
#   df = remove_anomalies(df) ~ Removing the anaomalies might break the model

# get the numeric columns using the get_numeric_columns function from preprocess_data.py
numeric_columns = get_numeric_columns(df)
# iterative outlier removal using the iterative_outlier_removal function from preprocess_data.py
df = iterative_outlier_removal(df, numeric_columns)
# make the data one-hot
df = make_Onehot(df)
# Model preparation
X, y = split_features_target(df)
# create an train test plit
X_train, X_test, Y_train, Y_test = create_train_test_split(X, y)
# reloading and training the deeplearning model
model=get_model('xgboost')
# model training and evaliation
train_model(model, 'xgboost', X_train, X_test, Y_train, Y_test)

<>:38: SyntaxWarning: invalid escape sequence '\d'
<>:38: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Karl\AppData\Local\Temp\ipykernel_13752\2333588591.py:38: SyntaxWarning: invalid escape sequence '\d'
  df = load_data('..\data\Student_performance_data.csv')
c:\Users\Karl\source\repos\ApexCareSolutions\MLG382_Guided_Project_GroupX\src\preprocess_data.py:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({False: 0.0, True: 1.0})
C:\Users\Karl\AppData\Local\Temp\ipykernel_13752\2333588591.py:38: SyntaxWarning: invalid escape sequence '\d'
  df = load_data('..\data\Student_performance_data.csv')


AttributeError: 'super' object has no attribute '__sklearn_tags__'